In [1]:
import time
import requests
from bs4 import BeautifulSoup as bs
import lxml
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from collections import Counter
import csv

keyword_list = []
with open('SearchList.csv', 'rt') as f:
    reader=csv.reader(f)
    keyword_list = list(reader)

print("인식된 키워드 목록입니다 : ", keyword_list)

counts=Counter()
driver = webdriver.Chrome('./chromedriver')

CafeTitle = []
KEYWORD = []
Cafe_URL = []
Category_Name = []
NaverID = []
PostTitle = []
The_Number_of_Views = []
The_Number_of_Keyword = []
The_Number_of_Video = []
The_Number_of_Picture = []
The_Number_of_Comment = []
The_Number_of_Characters = []
TNOC_TEMP=[]
TNOK_TEMP=[]
CMMT_TEMP=[]
CN_TEMP=[]
CafeTag = []
Posting_Date = []
ImgSource=[]

print("카페 검색결과에서 주소를 확보하겠습니다!")
#카페 검색결과에서 카페 주소를 먼저 확보합니다
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=article&query={}'.format(keyword)
    html = requests.get(url)
    soup = bs(html.text, 'html.parser')

    
    for WHOLE in soup.findAll("a", {"class": "sh_cafe_title"}):
        WHOLE = str(WHOLE)
        url_first = WHOLE.find('href=')
        url_last = WHOLE.find('onclick')
        WHOLE = WHOLE[url_first+6:url_last-2]
        Cafe_URL.append(WHOLE)
        KEYWORD.append(keyword)

        
index = 0


KEYWORD = sum(KEYWORD, [])    

print("확보한 주소로 카페 정보를 가져오겠습니다!")
        
for URL in Cafe_URL:
    EnterSearch = "http://search.naver.com/search.naver?ie=UTF-8&url={}".format(URL)
    driver.get(EnterSearch)
    time.sleep(1.5)

    soup = bs(driver.page_source, 'html.parser')

    #글 내용의 정보를 가져오기 전에 카페 이름을 가져옵니다.
    CafeTitle.append(soup.select('body>h1')[0].get_text())
#    print(CafeTitle)
    
    # 글이 있는 frame으로 변경
    driver.switch_to.frame('cafe_main')
    soup = bs(driver.page_source, 'html.parser')
    
    # 게시글에서 제목 추출
    PostTitle.append(soup.select('div.tit-box span')[0].get_text())
#    print(PostTitle)

    # 게시글에서 날짜 추출
    Posting_Date.append(soup.select('tbody > tr > td.m-tcol-c.date')[0].get_text())
#    print(Posting_Date)
    
    # 게시글에서 닉네임(ID) 추출
    NaverID.append(soup.select('td.p-nick > a')[0].get_text())
#    print(NaverID)

    # 카테고리명 추출
    CN_TEMP.append(soup.select('div > div.tit-box > div.fl > table > tbody > tr > td')[0].get_text())
    for Category in CN_TEMP:
        Category_R=Category.replace(" ", "").replace("\n", "").replace("\xa0", "").replace("\t", "").replace("-", "")
    Category_Name.append(Category_R)
#    print(Category_Name)

    # 조회수 추출
    The_Number_of_Views.append(soup.select('#cmtMenu > div.fl.reply_sort > table > tbody > tr > td:nth-child(5) > span:nth-child(2)')[0].get_text())
#    print(The_Number_of_Views)

    # 댓글수 추출
    CMMT_TEMP.append(soup.select('#comment')[0].get_text())
    for Comments in CMMT_TEMP:
        Comments_R=Comments.replace(" ", "").replace("\n", "").replace("댓글", "").replace("\t", "").replace("-", "")
    The_Number_of_Comment.append(Comments_R)
#    print(The_Number_of_Comment)
    
    # 태그종류 추출- 태그가 없는 경우 아무것도 입력되지 않습니다!
    CafeTag.append(soup.select('div > table.tag_n_id.tag_list_area > tbody > tr > td')[0].get_text())
#    print(CafeTag)

    # 게시글내 이미지수를 파악합니다.
    ImgSearch=driver.find_elements_by_name("cafeuserimg")
    The_Number_of_Picture.append(len(ImgSearch))
#    print(The_Number_of_Picture)

    # 게시글내 동영상수를 파악합니다.
    iframeCount=driver.find_elements_by_xpath('//*[@id="tbody"]//iframe')
    The_Number_of_Video.append(len(iframeCount))

    # 글자수 파악
    TNOC_TEMP.append(soup.select('#tbody')[0].get_text())
    
    KEYWORD_COUNT = soup.select('#tbody')[0].get_text()
    
    TNOK_TEMP=TNOC_TEMP
    for LETTERS in TNOC_TEMP:
        LETTERS_B=LETTERS.replace(" ", "").replace("\n", "").replace("\xa0", "").replace("\t", "").replace("-", "")
    The_Number_of_Characters.append(len(LETTERS_B))

    count_K = KEYWORD_COUNT.count(KEYWORD[index])
    The_Number_of_Keyword.append(count_K)
    index += 1


driver.close()

print("데이터 수집 완료.. 카페 데이터를 출력합니다!")
    
Cafe_dataframe = pd.DataFrame({'키워드' : KEYWORD,
                                '카페명' : CafeTitle,
                                '카페 주소' : Cafe_URL,
                                '카테고리명' : Category_Name,
                                '글쓴이ID' : NaverID,
                                '포스트 제목' : PostTitle,
                                '포스트일' : Posting_Date,
                                '조회수' : The_Number_of_Views,
                                '비디오수' : The_Number_of_Video,
                                '그림수' : The_Number_of_Picture,
                                '댓글수' : The_Number_of_Comment,
                                '글자수' : The_Number_of_Characters,
                                '키워드수' : The_Number_of_Keyword
                               })

Cafe_dataframe.to_csv('cafe_data.csv', index=False)

print("카페 데이터 출력을 완료했습니다!")



인식된 키워드 목록입니다 :  [['고양이'], ['강아지'], ['개'], ['헤어']]
카페 검색결과에서 주소를 확보하겠습니다!
확보한 주소로 카페 정보를 가져오겠습니다!
데이터 수집 완료.. 카페 데이터를 출력합니다!
카페 데이터 출력을 완료했습니다!


In [2]:
MonS_PC = []
MonS_MO = []
MonCN_PC = []
MonCN_MO = []
MonCR_PC = []
MonCR_MO = []
CMPT = []
EXPS = []

driver = webdriver.Chrome('./chromedriver')

print("실전광고 홈페이지 검색을 시작합니다!")

for keyword in keyword_list:
    siljun_url= "https://www.af-ad.co.kr/napi/form.php"
    driver.get(siljun_url)
    time.sleep(1)
    Send_Keyword=driver.find_element_by_xpath('//*[@id="keywords"]')
    Send_Keyword.send_keys(keyword)
    click_search=driver.find_element_by_xpath('//*[@id="sh_content"]/div/form/div/ul/li[3]/input')
    click_search.click()
    
    MonS_PC_S = driver.find_elements_by_css_selector('#resultform > tbody > tr:nth-child(3) > td:nth-child(2)')
    for value in MonS_PC_S:
        MonS_PC.append(value.text)
            
    MonS_MO_S = driver.find_elements_by_css_selector('#resultform > tbody > tr:nth-child(3) > td:nth-child(3)')
    for value in MonS_MO_S:
        MonS_MO.append(value.text)
        
    
    MonCN_PC_S = driver.find_elements_by_css_selector('#resultform > tbody > tr:nth-child(3) > td:nth-child(4)')
    for value in MonCN_PC_S:
        MonCN_PC.append(value.text)
        
    
    MonCN_MO_S = driver.find_elements_by_css_selector('#resultform > tbody > tr:nth-child(3) > td:nth-child(5)')
    for value in MonCN_MO_S:
        MonCN_MO.append(value.text)
        
        
    MonCR_PC_S = driver.find_elements_by_css_selector('#resultform > tbody > tr:nth-child(3) > td:nth-child(6)')
    for value in MonCR_PC_S:
        MonCR_PC.append(value.text)
        
        
    MonCR_MO_S = driver.find_elements_by_css_selector('#resultform > tbody > tr:nth-child(3) > td:nth-child(7)')
    for value in MonCR_MO_S:
        MonCR_MO.append(value.text)
        
    
    CMPT_S = driver.find_elements_by_css_selector('#resultform > tbody > tr:nth-child(3) > td:nth-child(8)')
    for value in CMPT_S:
        CMPT.append(value.text)
        
        
    EXPS_S = driver.find_elements_by_css_selector('#resultform > tbody > tr:nth-child(3) > td:nth-child(9)')
    for value in EXPS_S:
        EXPS.append(value.text)

driver.close()        

print("데이터 수집 완료.. 실전광고 데이터를 출력합니다!")        
    
Siljeon_dataframe = pd.DataFrame({'키워드' : keyword_list,
                                  '월간검색수PC' : MonS_PC,
                                  '월간검색수MO' : MonS_MO,
                                  '월평균클릭수PC' : MonCN_PC,
                                  '월평균클릭수MO' : MonCN_MO,
                                  '월평균클릭률PC' : MonCR_PC,
                                  '월평균클릭률MO' : MonCR_MO,
                                  '경쟁정도' : CMPT,
                                  '월평균노출\n광고수' : EXPS
                               })

Siljeon_dataframe.to_csv('Siljeon_data.csv', index=False)

print("실전광고 데이터 출력을 완료했습니다!")



실전광고 홈페이지 검색을 시작합니다!
데이터 수집 완료.. 실전광고 데이터를 출력합니다!
실전광고 데이터 출력을 완료했습니다!


In [3]:
Cafe_dataframe

,키워드,카페명,카페 주소,카테고리명,글쓴이ID,포스트 제목,포스트일,조회수,비디오수,그림수,댓글수,글자수,키워드수
0,고양이,리그오브레전드 한국커뮤니티 - LoLKor,http://cafe.naver.com/lolkor/20378693,벵갈고양이에대해알아보자,관심병 전용사료(gusw****),벵갈 고양이에 대해 알아보자,2019.02.15. 14:20,"1,777",0,9,32,1230,10
1,고양이,"냥이네-고양이를 사랑하는 모임,길 고양이 무료분양,길냥이 분양",http://cafe.naver.com/clubpet/3679598,[대구],수미맘(cute****),[대구],2019.02.23. 23:55,132,0,4,12,6062,9
2,고양이,진주아지매,http://cafe.naver.com/jinjululu/1241784,태국왕실고양이~샴!!!,달님들맘74(kang****),태국 왕실 고양이~샴!!!,2019.02.17. 13:40,597,0,4,11,613,4
3,고양이,"고사모-고양이를 사랑하는 모임,길 고양이 무료분양 길냥이 캣맘",http://cafe.naver.com/catmam/781859,42년인생에서첨으로고양이를키워봅니다.,이편한 아빠딸(1099****),42년 인생에서 첨으로 고양이를 키워봅니다.,2019.02.21. 17:11,318,0,8,18,1540,8
4,고양이,"관악맘블리♡- 관악,금천,구로,동작맘 대표맘까페",http://cafe.naver.com/news2080/136054,고양이까페,영은가은맘v삼성동(jigu****),고양이까페,2019.02.20. 00:17,251,0,6,13,153,3
5,고양이,"길고양이 급식소-길냥이 캣맘 모임,고양이 구조,유기묘 무료분양",http://cafe.naver.com/caretaker/111405,부산고양이분양합니다.ㅠㅠ,도도돋(tkqm****),부산 고양이 분양합니다. ㅠㅠ,2019.02.18. 23:16,927,0,23,20,705,12
6,고양이,"""하이디Cafe""그릇앤틱코펜하겐스타우브마담알렉산더유럽구매대행",http://cafe.naver.com/heidicafe/1219376,(드림)고양이는알고있지,M Flora(nico****),(드림)고양이는 알고 있지,2019.02.18. 16:03,135,0,6,17,334,4
7,고양이,일본맘,http://cafe.naver.com/japanmama/325497,처음으로고양이카페다녀왔어요~,수퍼우먼(sian****),처음으로 고양이카페 다녀왔어요~,2019.02.19. 08:47,245,0,14,12,452,4
8,고양이,"우마동- 강아지,고양이무료분양,애완동물,고슴도치,열대어분양",http://cafe.naver.com/6655happyclub/45129,스코티쉬폴드고양이무료분양순하다냥,반려동물사랑해(caca****),스코티쉬폴드 고양이무료분양 순하다냥,2019.02.19. 13:35,"1,664",1,8,10,544,11
9,고양이,빈티지마을,http://cafe.naver.com/vt7/23871,양육중인대형고양이forAltec,이규영(roma****),양육중인 대형 고양이 for Altec,2019.02.19. 17:05,609,0,1,10,656,4


In [4]:
Siljeon_dataframe

,키워드,월간검색수PC,월간검색수MO,월평균클릭수PC,월평균클릭수MO,월평균클릭률PC,월평균클릭률MO,경쟁정도,월평균노출 광고수
0,[고양이],"35,900","126,000",127.7,2856.7,0.38%,2.44%,높음,15
1,[강아지],"31,100","124,800",140.5,2770.7,0.48%,2.36%,높음,15
2,[개],"5,600","31,900",3.3,10,0.07%,0.04%,높음,3
3,[헤어],"2,530","17,500",1.8,2.7,0.08%,0.02%,높음,15
